<a href="https://colab.research.google.com/github/dxda6216/q10/blob/main/circadian_period_q10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import statistics
import pandas as pd
from scipy.optimize import curve_fit
from matplotlib import pyplot as plt

#@markdown For usage, see [GitHub repository page](https://github.com/dxda6216/q10).

#@markdown Input data, then hit **Runtime** -> **Run all** (or press **Ctrl+F9**).

# Helper function to print statistics
def print_statistics(data_array, name, unit):
    min_val = np.min(data_array)
    max_val = np.max(data_array)
    avg_val = statistics.mean(data_array)
    med_val = statistics.median(data_array)
    print(f"Minimum {name} = {min_val:.3f} {unit}")
    print(f"Maximum {name} = {max_val:.3f} {unit}")
    print(f"Average {name} = {avg_val:.3f} {unit}")
    print(f"Median {name}  = {med_val:.3f} {unit}\n")

# Data description (plot title)
Data_description = "Mutant ABC" #@param {type:"string"}

# Temperature data
Temperatures = 25.1, 26.9, 27.3, 29.2, 30, 31.7, 32.2, 33.5, 34.1, 35.3, 35.8, 36.5, 37.1, 38.5, 38.9, 39.3, 40.3, 41.1 #@param {type:"raw"}
x = np.array(Temperatures)

# Period data
Periods =  33.6, 32.8, 30.4, 29.3, 28.0, 27.9, 26.1, 25.5, 24.9, 24.8, 24.5, 24.4, 24.1, 23.9, 23.6, 21.9, 21.5, 20.1 #@param {type:"raw"}
y = np.array(Periods)

# Creating Pandas dataframe (df) for the input data
tp = {'Temperature': x, 'Period': y}
df = pd.DataFrame(data=tp)

# Printing data
print("Input data")
print(df, "\n")

print_statistics(x, "temperature", "\u00B0C")
print_statistics(y, "period", "h")

# Calculation Q10 from the data within a specific temperature range
#@markdown Only data within a specific temperature range to be used for Q10 calculation? (Yes or No)
Set_temperature_range = "No" #@param ["Yes (set lowest and highest limits by sliders below)", "No"]
Range_low_limit = 15 # @param {type:"slider", min:0, max:65, step:0.1}
Range_high_limit = 50 # @param {type:"slider", min:0, max:65, step:0.1}

# Creating a new dataframe (dfa) for a data set within the temperature range
if Set_temperature_range == "No":
    dfa = df
else:
    if Range_low_limit < Range_high_limit:
        dfa = df[(df['Temperature'] >= Range_low_limit) & (df['Temperature'] <= Range_high_limit)]
    else:
        dfa = df

xa = dfa['Temperature']
ya = dfa['Period']

# Printing the data set for Q10 calculation
print("Data used for Q10 calculation")
print(dfa, "\n")

print_statistics(xa, "temperature", "\u00B0C")
print_statistics(ya, "period", "h")

# Base temperature
#@markdown Setting base temperature
Select_base_temperature = "37°C" #@param ["-273.15\u00B0C (absolute zero)", "0\u00B0C", "4\u00B0C", "25\u00B0C", "30\u00B0C", "37\u00B0C", "100\u00B0C", "Minimum", "Maximum", "Average", "Median", "Set \"Base_temperature\" by slider below"]
Base_temperature = 37 # @param {type:"slider", min:0, max:100, step:0.1}

base_t_mapping = {
    "-273.15\u00B0C (absolute zero)": -273.15,
    "0\u00B0C": 0.000,
    "4\u00B0C": 4.000,
    "25\u00B0C": 25.000,
    "30\u00B0C": 30.000,
    "37\u00B0C": 37.000,
    "100\u00B0C": 100.000,
    "Minimum": np.min(xa),
    "Maximum": np.max(xa),
    "Average": statistics.mean(xa),
    "Median": statistics.median(xa),
    "Set \"Base_temperature\" by slider below": Base_temperature
}

base_t = base_t_mapping.get(Select_base_temperature, Base_temperature)

# Displaying Tab-delimited data Yes or No
#@markdown Displaying tab-delimited data? (Yes or No)
Display_tab_delimited_data = "No" #@param ["Yes", "No"]

# Defining an equation for curve fitting
def func(xa, tau_bt, q10):
    return tau_bt * q10 ** ( ( base_t - xa ) * 0.1 )

# Initial values for the fitting parameters
p0 = np.array([24.000, 1.000])

# Fitting the data to the defined equation
popt, pcov = curve_fit(func, xa, ya, p0)
residuals = ya - func(xa, *popt)
ss_residuals = np.sum(residuals**2)
ss_total = np.sum((ya-np.mean(ya))**2)
r_squared = 1 - ( ss_residuals / ss_total )

# Setting up fig
fig = plt.figure(figsize = (8,6))
fcxmin = int( min(x) - ( max(x) - min(x) ) * 0.333 )
fcxmax = int( max(x) + ( max(x) - min(x) ) * 0.333 ) + 1
fcx = np.linspace(fcxmin, fcxmax, 200)

if Set_temperature_range == "No":
  plt.scatter(xa, ya, marker='o', s=35, color ='red', label ='Data')
else:
  plt.scatter(x, y, marker='o', s=30, color ='lightgreen', label ='Data')
  plt.scatter(xa, ya, marker='o', s=35, color ='red', label ='Data points used for Q10 calculation')

fcy = func(fcx, popt[0], popt[1])
plt.plot(fcx, fcy, '--', color='blue', label =f'Fit        Q10 = {popt[1]:5.3f}')
plt.xlabel(u'Temperature (\u00B0C)')
plt.ylabel('Period (hours)')

# Displaying tab delimited data
if Display_tab_delimited_data == "Yes":
    print('Dataset')
    print ('Temp (\u00B0C)\tPeriod (hours)')
    for i in range(len(x)):
        print(f"{x[i]:.3f}\t{y[i]:.3f}") # Added .3f for consistency in period output
    print('\nFitted Curve')
    print ('Temp (\u00B0C)\tPeriod (hours)')
    for i in range(len(fcx)):
        print(f"{fcx[i]:.3f}\t{fcy[i]:.3f}")
    print('\n')

# Printing the results
print(f'Estimated period length at {base_t:.2f} \u00B0C = {popt[0]:.3f} \u00B1 {pcov[0,0]**0.5:.3f} hours')
print(f'Q10 (temperature coefficient) = {popt[1]:.3f} \u00B1 {pcov[1,1]**0.5:.3f}')
print(f'R\u00B2 = {r_squared:.6f}\n')

# Adjustment of plot location
plt.subplots_adjust(top=0.9, bottom=0.0)

# Fig title, header, footer
fig.suptitle(Data_description, fontsize=12)
fst = f'Eest. tau at {base_t:.2f} \u00B0C = {popt[0]:.3f} \u00B1 {pcov[0,0]**0.5:.3f} hrs      Q10 = {popt[1]:.3f} \u00B1 {pcov[1,1]**0.5:.3f}'
fig.text(0.5, 0.92, fst, horizontalalignment="center", fontsize=9)
plt.legend()

# Showing fig
plt.show()